# Session

In [ ]:
#| default_exp session

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import csv

import os
import re
from pathlib import Path
from typing import Tuple, List, Union, Optional, Callable, Type, Iterable, no_type_check, Sequence
from IPython import display

import pandas as pd
import os

import logging
logger = logging.getLogger(__name__)

from spannerlib.utils import checkLogs,serialize_df_values,get_base_file_path
from spannerlib.grammar import parse_spannerlog,reconstruct
from spannerlib.span import Span
from spannerlib.data_types import (
    _infer_relation_schema,
     Var,
    FreeVar,
    RelationDefinition,
    Relation,
    IEFunction,
    IERelation,
    Rule,
    pretty,
)
from spannerlib.engine import Engine

from spannerlib.micro_passes import (
    convert_primitive_values_to_objects,
    remove_new_lines_from_strings,
    CheckReservedRelationNames,
    check_referenced_paths_exist,
    dereference_vars,
    relations_to_dataclasses,
    verify_referenced_relations,
    rules_to_dataclasses,
    consistent_free_var_types_in_rule,
    check_rule_safety,
    assignments_to_name_val_tuple,
    execute_statement,
)


In [ ]:
#| export
#| hide


# from spannerlib.ie_func.json_path import JsonPath, JsonPathFull
# from spannerlib.ie_func.nlp import (Tokenize, SSplit, POS, Lemma, NER, EntityMentions, CleanXML, Parse, DepParse, Coref, OpenIE, KBP, Quote, Sentiment, TrueCase)
from spannerlib.ie_func.python_regex import PYRGX, PYRGX_STRING
# from spannerlib.ie_func.rust_spanner_regex import RGX, RGX_STRING, RGX_FROM_FILE, RGX_STRING_FROM_FILE


# # ordered by rgx, json, nlp, etc.
# PREDEFINED_IE_FUNCS = [PYRGX, PYRGX_STRING, RGX, RGX_STRING, RGX_FROM_FILE, RGX_STRING_FROM_FILE,
#                        JsonPath, JsonPathFull,
#                        Tokenize, SSplit, POS, Lemma, NER, EntityMentions, CleanXML, Parse, DepParse, Coref, OpenIE, KBP, Quote, Sentiment,
#                        TrueCase]





## Session class

In [ ]:
#| export
class Session():
    def __init__(self):
        
        self.pass_stack = [
            convert_primitive_values_to_objects,
            remove_new_lines_from_strings,
            CheckReservedRelationNames('spanner_'),
            check_referenced_paths_exist,
            dereference_vars,
            relations_to_dataclasses,
            verify_referenced_relations,
            rules_to_dataclasses,
            consistent_free_var_types_in_rule,
            check_rule_safety,
            assignments_to_name_val_tuple
        ]
        self.engine=Engine()

        # TODO define all the default ie funcs

    def get_pass_stack(self):
        """
        @return: the current pass stack.
        """
        return self._pass_stack.copy()

    def set_pass_stack(self,
        user_stack #  a user supplied pass stack
        ): 
        """
        Sets a new pass stack instead of the current one.
        """
        if type(user_stack) is not list:
            raise TypeError('user stack should be a list of passes')
        self._pass_stack = user_stack.copy()

    def _display_result(self,result):
        if result is None:
            pass
        elif isinstance(result,pd.DataFrame):
            display.display(result)
        else:
            print(result)
    
    def register(self,name,func,in_schema,out_schema):
        #TODO add option for outschema to be a callable that gets the input and the output and confirms if the output is valid
        ie_func_obj = IEFunction(name=name,func=func,in_schema=in_schema,out_schema=out_schema)
        self.engine.set_ie_function(ie_func_obj)

    def parse_and_check_semantics(self,code):
        statements = parse_spannerlog(code,split_statements=True)
        asts = []
        for statement_nx,statement_lark in statements:
            ast = statement_nx
            for pass_ in self.pass_stack:
                try:
                    pass_(ast,self.engine)
                except Exception as e:
                    raise Exception(
                        f"During semantic checks for statement \n\"{reconstruct(statement_lark)}\"\n"
                        f"in pass {pass_} the following exception was raised:\n{e}\n"
                        ).with_traceback(e.__traceback__)
            yield ast,statement_lark

    def handle_boolean_results(self,res):
        if not isinstance(res,pd.DataFrame):
            return res
        if res.shape == (1,0):
            return True
        elif res.shape == (0,0):
            return False
        else:
            return res

    def plan_query(self,code):
        statements = list(self.parse_and_check_semantics(code))
        if len(statements) > 1:
            raise ValueError(f"Only one statement is allowed in plan_query, got {len(statements)}")
        ast,_ = statements[0]
        statement_node = list(ast.nodes)[0]
        node_data = ast.nodes[statement_node]
        statement = node_data['type']
        value = node_data['val']
        if statement != 'query':
            raise ValueError(f"Expected a query statement, got {statement}")

        query_graph,root =  self.engine.plan_query(value)
        return query_graph,root

    def execute_plan(self,query_graph,root,return_intermediate=False):
        res,inter = self.engine.execute_plan(query_graph,root,return_intermediate=True)
        res = self.handle_boolean_results(res)
        if return_intermediate:
            return res,inter
        return res
        

    def export(self,code,display_results=False):
        #TODO reconstruct the code for each statement using lark,reconstruct so we can print the query string together with the result
        results = []
        for clean_ast,statement_lark in self.parse_and_check_semantics(code):
            try:
                result = execute_statement(clean_ast,self.engine)
                result = self.handle_boolean_results(result)
            except Exception as e:
                raise Exception(
                    f"During execution of statement \n\"{reconstruct(statement_lark)}\n\""
                    f"the following exception was raised:\n{e}\n"
                    ).with_traceback(e.__traceback__)
            results.append(result)
            if display_results:
                self._display_result(result)
        return results[-1]

    def import_rel(self,name:str,data:Union[str,Path,pd.DataFrame],delim:str = None):
        """Imports a relation into the current session, either from a dataframe or from a csv file."""
        if isinstance(data, (Path,str)):
            csv_file_name = Path(data)
            if not csv_file_name.is_file():
                raise IOError("csv file does not exist")
            if os.stat(csv_file_name).st_size == 0:
                raise IOError("csv file is empty")
            data = pd.read_csv(csv_file_name, delimiter=delim)

        first_row = list(data.iloc[0,:])
        scheme = _infer_relation_schema(first_row)
        rel_def = RelationDefinition(name=name,scheme=scheme)
        self.engine.set_relation(rel_def)
        self.engine.add_facts(name,data)
        
    def print_rules(self):
        rules = list(self.engine.rules_to_ids.keys())
        for rule in rules:
            print(rule)
        return rules
    def remove_rule(self,rule:str):
        self.engine.del_rule(rule)
    def remove_relation(self,relation:str):
        self.engine.del_relation(relation)
    def clear(self):
        self.engine = Engine()

    def get_all_functions(self):
        return self.engine.ie_functions.copy()

## Examples

In [ ]:
commands = """
    new Parent(str, str)
    Parent("Sam", "Noah")
    Parent("Noah", "Austin")
    Parent("Austin", "Stephen")

    GrandParent(G, C) <- Parent(G, M), Parent(M, C)
    ?GrandParent(X, "Austin")
    """
session = Session()
res = session.export(commands,display_results=True)
assert serialize_df_values(res) == {('Sam',)}

# check that rerunning query gives the same result
res = session.export('?GrandParent(X, "Austin")')
assert serialize_df_values(res) == {('Sam',)}

,X
0,Sam


In [ ]:
def length(string: str) -> Iterable[int]:
        yield (len(string),)

sess = Session()
sess.register('Length',length,[str],[int])
res = sess.export('''
    new string(str)
    string("a")
    string("d")
    string("a")
    string("ab")
    string("abc")
    string("abcd")

    string_length(Str, Len) <- string(Str), Length(Str) -> (Len)
    # ?string_length(Str, Len)
''')
#TODO from here, add the rest of the examples from the old notebook and start moving the failed ones to tests in the respective passes/execution
res

In [ ]:
from graph_rewrite import draw

In [ ]:
q,root = sess.plan_query("?string_length(Str, Len)")
# draw(q)
# with checkLogs(name='spannerlib.engine'):
res,inter = sess.execute_plan(q,root,return_intermediate=True)
assert serialize_df_values(res) == {('a', 1), ('ab', 2), ('abc', 3), ('abcd', 4), ('d', 1)}
res

,Str,Len
0,a,1
2,d,1
5,ab,2
6,abc,3
7,abcd,4


In [ ]:
# test rule removal

sess = Session()
_ = sess.export('''
    new parent(str, str)
    new grandparent(str, str)
    parent("Liam", "Noah")
    parent("Noah", "Oliver")
    parent("James", "Lucas")
    parent("Noah", "Benjamin")
    parent("Benjamin", "Mason")
    grandparent("Tom", "Avi")
    ancestor(X,Y) <- parent(X,Y)
    ancestor(X,Y) <- grandparent(X,Y)
    ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
''')


rules = sess.print_rules()
assert rules == ['ancestor(X,Y) <- parent(X,Y)',
'ancestor(X,Y) <- grandparent(X,Y)',
'ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)',]

sess.remove_rule("ancestor(X,Y) <- parent(X,Y)")
print("="*50)
rules = sess.print_rules()
assert rules == ['ancestor(X,Y) <- grandparent(X,Y)',
'ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)',]

ancestor(X,Y) <- parent(X,Y)
ancestor(X,Y) <- grandparent(X,Y)
ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)
ancestor(X,Y) <- grandparent(X,Y)
ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)


In [ ]:
# test clearing the engine
commands = """
    new parent(str, str)
    new grandparent(str, str)
    parent("Liam", "Noah")
    grandparent("Tom", "Avi")
    ancestor(X,Y) <- parent(X,Y)
    ancestor(X,Y) <- grandparent(X,Y)
    ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
    """
session = Session()
output = session.export(commands)
session.print_rules()
session.clear()
assert session.print_rules() == []

ancestor(X,Y) <- parent(X,Y)
ancestor(X,Y) <- grandparent(X,Y)
ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)


In [ ]:
# importing relations from csv
session = Session()
session.import_rel(name="enrolled",data="./sample_data/enrolled.csv", delim=",")
commands = """
enrolled("abigail", "chemistry")
?enrolled(X,Y)
"""
res = session.export(commands)
assert serialize_df_values(res) == {('abigail', 'chemistry'),
 ('gale', 'operating_systems'),
 ('howard', 'chemistry'),
 ('howard', 'physics'),
 ('jordan', 'chemistry')}
res

,X,Y
0,jordan,chemistry
1,gale,operating_systems
2,howard,chemistry
3,howard,physics
0,abigail,chemistry


In [ ]:
# # use standard ie functions
# # TODO fix this
# session = Session()
# session.import_rel(name="enrolled",data="./sample_data/enrolled.csv", delim=",")
# commands = """
#     enrolled("abigail", "chemistry")
# gpa_str = "abigail 100 jordan 80 gale 79 howard 60"

# gpa(Student,Grade) <- py_rgx_string(gpa_str, "(\w+).*?(\d+)")->(Student, Grade),enrolled(Student,X)

# ?gpa(X,Y)
# """
# x = session.export(commands)

In [ ]:
# importing relations from dataframe
session = Session()
lecturer_df = pd.DataFrame(([["walter","chemistry"], ["linus", "operating_systems"]]))
session.import_rel("lecturer",lecturer_df)
commands = """ 
?lecturer(X,Y)
"""
res = session.export(commands)
assert serialize_df_values(res) == {('linus', 'operating_systems'), ('walter', 'chemistry')}
res

,X,Y
0,walter,chemistry
1,linus,operating_systems


In [ ]:
# empty input to ie functions
commands = """
        new A(str, str)
        new B(str, str)
        A("1", "2")
        B("1", "3_id")
        C(X, Y) <- A(X, Y)
        D(X, Y, X) <- C(X, Y)
        # nothing will feed into ID but we still need output in the same schema as the first D rule
        D(X, Y, Z) <- A(X, "1"), B(X, Y), ID(X) -> (Y), ID2(Y)->(Z,W)
        ?D(X, Y, Z)
    """

def ID(string: str):
        # here we append the input to the output inside the ie function!
        yield f'{string}_id'

def ID2(string: str):
        # here we append the input to the output inside the ie function!
        yield f'{string}_id2_z',f'{string}_id2_w'

session = Session()
session.register('ID',ID, [str], [str])
session.register('ID2',ID2, [str], [str,str])
# with checkLogs(name='spannerlib.engine'):
res = session.export(commands)
#TODO form here
assert serialize_df_values(res)=={('1', '2', '1')}
res


,X,Y,Z
0,1,2,1


In [ ]:
# multiple ie functions
commands = """
        new A(str, str)
        new B(str, str)
        A("1", "2")
        B("1", "1_id")
        C(X, Y) <- A(X, Y)
        D(X, Y, X) <- C(X, Y)
        # nothing will feed into ID but we still need output in the same schema as the first D rule
        D(X, Y, Z) <- A(X, "2"), B(X, Y), ID(X) -> (Y), ID2(Y)->(Z,W)
        # ?D(X, Y, Z)
    """

def ID(string: str):
        # here we append the input to the output inside the ie function!
        yield (f'{string}_id',)

def ID2(string: str):
        # here we append the input to the output inside the ie function!
        yield f'{string}_id2_z',f'{string}_id2_w'

session = Session()
session.register('ID',ID, [str], [str])
session.register('ID2',ID2, [str], [str,str])
# with checkLogs(name='spannerlib.engine'):

res = session.export(commands)
#TODO form here
# assert serialize_df_values(res)=={('1', '2', '1')}
res


In [ ]:
q,root= session.plan_query("?D(X, Y, Z)")
# draw(q)
# with checkLogs(name='spannerlib.engine'):
res = session.execute_plan(q,root)
assert serialize_df_values(res) == {('1', '1_id', '1_id_id2_z'), ('1', '2', '1')}
res

,X,Y,Z
0,1,2,1
1,1,1_id,1_id_id2_z


In [ ]:
"he ho he ho".split()

['he', 'ho', 'he', 'ho']

In [ ]:
# ie functions bounded by constants
commands = """
    new String(str)
    String("he")
    String("hehe")
    Text(T) <- Split("he ho hehe hoho")->(T)
    StringFromText(S) <- String(S), Split("he ho hehe hoho")->(S)
    """
session = Session()

def split(string: str):
    for part in string.split():
        yield (part,)

session.register('Split',split, [str], [str])
session.export(commands)
# assert serialize_df_values(res) == {('1', '1_id', '1_id_id2_z'), ('1', '2', '1')}
q,_ = session.plan_query("?StringFromText(S)")
draw(q)
res = session.export("?StringFromText(S)")
res

,S
0,he
1,hehe


In [ ]:
q,root = session.plan_query("?Text(T)")
draw(q)
res = session.execute_plan(q,root)
assert serialize_df_values(res) == {('he',), ('hehe',), ('ho',), ('hoho',)}

In [ ]:
# more datalog tests
commands = """
    new Parent(str, str)
    Parent("Sam", "Noah")
    Parent("Noah", "Austin")
    Parent("Austin", "Stephen")

    GrandParent(G, C) <- Parent(G, M), Parent(M, C)
    ?GrandParent(X, "Austin")
    """


session = Session()
res = session.export(commands)
assert serialize_df_values(res) == {('Sam',)}
res
res = session.export("""?GrandParent("Sam", "Austin")""")
assert res == True

res = session.export("""?GrandParent("Sams", "Austin")""")
assert res == False


In [ ]:
# use of ie functions with variable output arity
commands = """
input_string = "John Doe: 35 years old, Jane Smith: 28 years old"
AgeOf(Name,Age) <- py_rgx_string(input_string, "(\w+\s\w+):\s(\d+)") -> (Name,Age)
?AgeOf(X,Y)
"""
session = Session()
session.register(*PYRGX_STRING)
res = session.export(commands)
assert serialize_df_values(res) == {('Jane Smith', '28'), ('John Doe', '35')}
res


,X,Y
0,John Doe,35
1,Jane Smith,28


In [ ]:
# use of ie functions with variable input arity
commands = """
new AgeOf(str, str)
AgeOf("John Doe", "35")
AgeOf("Jane Smith", "28")
age_description(Desc) <- AgeOf(Name, Age), format("{} is {} years old",Name,Age) -> (Desc)
?age_description(D)
"""
def format_ie(f_string,*params):
    yield f_string.format(*params),

string_schema = lambda x: ([str]*x)

session = Session()
session.register('format',format_ie, string_schema, [str])
res = session.export(commands)
assert serialize_df_values(res) == {('Jane Smith is 28 years old',), ('John Doe is 35 years old',)}
res


,D
0,John Doe is 35 years old
1,Jane Smith is 28 years old


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     